In [6]:
import numpy as np
import pandas as pd
from PIL import Image
import os 
import os.path
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [7]:

PATH = './labelled_images'
image_urls=[os.path.join(PATH, image) for image in os.listdir(PATH)]


In [8]:

img_example= cv2.imread(image_urls[0], cv2.IMREAD_UNCHANGED)


In [9]:

scale_percent = 10
width = int(img_example.shape[1] * scale_percent / 100)
height = int(img_example.shape[0] * scale_percent / 100)
dim = (width, height)


In [10]:

image_urls.remove("./labelled_images/.ipynb_checkpoints")


In [11]:

images_resized=[cv2.resize(cv2.imread(image_url, cv2.IMREAD_UNCHANGED), dim, interpolation = cv2.INTER_AREA) for image_url in image_urls]


In [12]:

image_array=np.array([cv2.cvtColor(image_resized, cv2.COLOR_BGR2RGB) for image_resized in images_resized])


In [13]:

image_labels=np.array([image_url.split('/')[-1].split('_')[1] for image_url in image_urls])


In [14]:

import re
image_labels_final=np.array([ 'working' if bool(re.search('.jpeg$', image_label)) else image_label for image_label in image_labels])
        

In [15]:

image_labels_final[image_labels_final == 'working'] = 0
image_labels_final[image_labels_final == 'crack'] = 1
image_labels_final[image_labels_final == 'lcd'] = 2
image_labels_final[image_labels_final == 'crackandlcd'] = 3


In [16]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(image_array, image_labels_final, test_size=0.3, random_state=4)


In [17]:

nsamples, nx, ny, nrgb = X_train.shape
X_train_resh = X_train.reshape(nsamples, nx*ny*nrgb)

nsamples_test, nx_test, ny_test, nrgb_test = X_test.shape
X_test_resh = X_test.reshape(nsamples_test, nx_test*ny_test*nrgb_test)



In [18]:

y_train=y_train.astype(int)
y_test=y_test.astype(int)


In [36]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

fsc1=0
fsc1s=0

rec=0
recs=0

accur=0
accurs=0

acc_list=[]

for i in range(10, 1010, 50):
    print("_____________________")
    print(i)
    RF = RandomForestClassifier(n_estimators= i)
    RF.fit(X_train_resh, y_train)
    predicted=RF.predict(X_test_resh)
    
    acc=round(RF.score(X_test_resh, y_test), 5)
    #print(f"Validation accuracy for {i} is {acc}%")
    acc_list.append(acc)
    
    recall=round(recall_score(y_test, predicted, average='macro'), 5)
    #print(f"Recall score for {i} is {recall}")
    
    
    f1=round(f1_score(y_test, predicted, average='macro'),5)
    #print(f"F1 score for {i} is {f1}")
    
    if f1 > fsc1:
        fsc1=f1
        fsc1s=i
    if acc > accur:
        accur=acc
        accurs=i
        
    if recall > rec:
        rec=recall
        recs=i
        

print("The best ones:")
print("Accuracy", accur, "at", accurs)
print("Recall", rec, "at", recs)
print("F1 score", fsc1, "at", fsc1s)


_____________________
10
_____________________
60
_____________________
110
_____________________
160
_____________________
210
_____________________
260
_____________________
310
_____________________
360
_____________________
410
_____________________
460
_____________________
510
_____________________
560
_____________________
610
_____________________
660
_____________________
710
_____________________
760
_____________________
810
_____________________
860
_____________________
910
_____________________
960
The best ones:
Accuracy 0.74699 at 260
Recall 0.50953 at 260
F1 score 0.50221 at 260
AAAAAA [0.6747, 0.6506, 0.66265, 0.71084, 0.71084, 0.74699, 0.73494, 0.68675, 0.72289, 0.72289, 0.73494, 0.71084, 0.72289, 0.72289, 0.71084, 0.72289, 0.71084, 0.71084, 0.6988, 0.74699]


In [37]:
print(acc_list)

[0.6747, 0.6506, 0.66265, 0.71084, 0.71084, 0.74699, 0.73494, 0.68675, 0.72289, 0.72289, 0.73494, 0.71084, 0.72289, 0.72289, 0.71084, 0.72289, 0.71084, 0.71084, 0.6988, 0.74699]


In [38]:

RF_optim = RandomForestClassifier(n_estimators= accurs)
RF_optim.fit(X_train_resh, y_train)


RandomForestClassifier(n_estimators=260)

In [40]:

predicted=RF_optim.predict(X_test_resh)
print(f"Accuracy at {accurs} estimators is {round(RF_optim.score(X_test_resh, y_test)*100, 2)}% \nRecall is {round(recall_score(y_test, predicted, average='micro')*100, 2)}%\nF1 score is {round(f1_score(y_test, predicted, average='macro')*100,2)}%")
                                                                           

Accuracy at 260 estimators is 77.11% 
Recall is 77.11%
F1 score is 51.79%


In [22]:

import xgboost as xgb
import sklearn


In [23]:

dtrain = xgb.DMatrix(X_train_resh, label=y_train)
dtest = xgb.DMatrix(X_test_resh, label=y_test)


In [25]:

n_rounds = 300
early_stopping = 4

highsc=0
nr=0

eval_list = [(dtrain, "train"), (dtest, "validation")]

for i in range(1, 41, 3):
    print("_________")
    print(i)
    param_list = [("objective", "multi:softmax"), ("eval_metric", "merror"), ("num_class", 3), ("max_depth", i)]
    bst = xgb.train(param_list, dtrain, n_rounds, evals=eval_list, early_stopping_rounds=early_stopping, verbose_eval=True)
    ypred = bst.predict(dtest)
    score=sklearn.metrics.accuracy_score(ypred, y_test)*100
    if score > highsc:
        highsc=score
        nr=i


_________
1
[0]	train-merror:0.30570	validation-merror:0.43373
[1]	train-merror:0.30052	validation-merror:0.38554
[2]	train-merror:0.29534	validation-merror:0.39759
[3]	train-merror:0.27979	validation-merror:0.38554
[4]	train-merror:0.26425	validation-merror:0.39759
[5]	train-merror:0.22798	validation-merror:0.37349
[6]	train-merror:0.22280	validation-merror:0.38554
[7]	train-merror:0.19689	validation-merror:0.36145
[8]	train-merror:0.17617	validation-merror:0.38554
[9]	train-merror:0.17098	validation-merror:0.36145
[10]	train-merror:0.16580	validation-merror:0.36145
[11]	train-merror:0.13472	validation-merror:0.33735
[12]	train-merror:0.13990	validation-merror:0.34940
[13]	train-merror:0.11399	validation-merror:0.36145
[14]	train-merror:0.09326	validation-merror:0.36145
[15]	train-merror:0.08290	validation-merror:0.33735
_________
4
[0]	train-merror:0.07254	validation-merror:0.37349
[1]	train-merror:0.03627	validation-merror:0.38554
[2]	train-merror:0.02591	validation-merror:0.37349
[

In [30]:
print(highsc, nr)

68.67469879518072 7


In [31]:
param_list = [("objective", "multi:softmax"), ("eval_metric", "mlogloss"), ("num_class", 3), ("max_depth", nr)]

n_rounds = 600

early_stopping = 15
eval_list = [(dtrain, "train"), (dtest, "validation")]

bst = xgb.train(param_list, dtrain, n_rounds, evals=eval_list, early_stopping_rounds=early_stopping, verbose_eval=True)

[0]	train-mlogloss:0.76394	validation-mlogloss:0.97735
[1]	train-mlogloss:0.55661	validation-mlogloss:0.89595
[2]	train-mlogloss:0.41797	validation-mlogloss:0.86860
[3]	train-mlogloss:0.32082	validation-mlogloss:0.82064
[4]	train-mlogloss:0.25192	validation-mlogloss:0.78278
[5]	train-mlogloss:0.19559	validation-mlogloss:0.77428
[6]	train-mlogloss:0.15603	validation-mlogloss:0.75058
[7]	train-mlogloss:0.12448	validation-mlogloss:0.72950
[8]	train-mlogloss:0.10173	validation-mlogloss:0.71121
[9]	train-mlogloss:0.08418	validation-mlogloss:0.71257
[10]	train-mlogloss:0.07121	validation-mlogloss:0.70578
[11]	train-mlogloss:0.06080	validation-mlogloss:0.70433
[12]	train-mlogloss:0.05225	validation-mlogloss:0.69569
[13]	train-mlogloss:0.04604	validation-mlogloss:0.70322
[14]	train-mlogloss:0.04081	validation-mlogloss:0.69897
[15]	train-mlogloss:0.03647	validation-mlogloss:0.70894
[16]	train-mlogloss:0.03261	validation-mlogloss:0.70658
[17]	train-mlogloss:0.02984	validation-mlogloss:0.69983
[1

In [32]:

ypred = bst.predict(dtest)
print(f"Accuracy at {nr} estimators is {round(sklearn.metrics.accuracy_score(y_test, ypred)*100, 2)}% \nRecall is {round(recall_score(y_test, ypred, average='macro')*100, 2)}%\nF1 score is {round(f1_score(y_test, ypred, average='macro')*100,2)}%")
                                            

Accuracy at 7 estimators is 67.47% 
Recall is 46.05%
F1 score is 45.27%
